In [1]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.portfolios

In [3]:
db.portfolios.find_one({'_id': '4ad3f122123d4c00a9c49fdae13b1dd4'})

{'_id': '4ad3f122123d4c00a9c49fdae13b1dd4',
 'name': 'porto2023',
 'created_at': datetime.datetime(2025, 5, 4, 11, 19, 54, 224000),
 'end_date': datetime.datetime(2023, 5, 4, 0, 0),
 'capital': 10000,
 'stocks': [{'ticker': 'CME',
   'price': 280.45,
   'weight': 0.10365943658757049,
   'allocated': 841.3499999999999,
   'shares': 3},
  {'ticker': 'CBOE',
   'price': 226.39,
   'weight': 0.09696982174248989,
   'allocated': 905.56,
   'shares': 4},
  {'ticker': 'KMB',
   'price': 130.35,
   'weight': 0.08871739711218611,
   'allocated': 782.0999999999999,
   'shares': 6},
  {'ticker': 'EA',
   'price': 151.1,
   'weight': 0.08148572085195978,
   'allocated': 755.5,
   'shares': 5},
  {'ticker': 'MCD',
   'price': 311.96,
   'weight': 0.07954139349805923,
   'allocated': 623.92,
   'shares': 2},
  {'ticker': 'GIS',
   'price': 55.43,
   'weight': 0.07331009744436356,
   'allocated': 720.59,
   'shares': 13},
  {'ticker': 'KO',
   'price': 71.65,
   'weight': 0.06908099251417722,
   'all

In [23]:
prices.set_index('ticker')['price'].get('A')

In [25]:
sorted(prices.set_index('ticker')['price'].items(), key=lambda x: x[1], reverse=True)

[('MSFT', 102), ('PLTR', 101), ('AAPL', 100)]

## FinViz stats

In [4]:
import aiohttp
import pandas as pd
from io import StringIO

from config import settings


async def get_ticker_info(tickers: list[str]):
    async with aiohttp.ClientSession() as session:
        data = await session.get(
            f'https://elite.finviz.com/export.ashx?v=152&t={",".join(tickers)}&o=-change&c=1,2,3,5,6,7,75,16,82,78,28,38,39,40,41,42,44,46,48,54,65,66&auth={settings.FINVIZ_API_KEY}'
        )
        df = pd.read_csv(
            StringIO(await data.text())
        ).set_index('Ticker')
        return df.to_dict('index')

def get_performance(dataset: pd.DataFrame, weights: dict):
    returns = dataset[weights.keys()].pct_change()
    portfolio_returns = returns.dot(pd.Series(weights))
    return ((1 + portfolio_returns).cumprod() * 100).dropna()

